In [2]:
import pandas as pd
import warnings

def warn(*args, **kwargs):
    pass
warnings.warn = warn

In [3]:
df = pd.DataFrame(columns=['image', 'label'])

In [4]:
sample8_ir = pd.read_csv(r'C:\Users\wasim\line-following\Data Samples\DataSamples-8\data_ir.csv')
sample8_images = pd.read_csv(r'C:\Users\wasim\line-following\Data Samples\DataSamples-8\out_images.csv')

In [5]:
sample8_ir

,timeStamp,IR1,IR2,ACTION
0,17/08/2022 02:06:16,1,1,FORWARD
1,17/08/2022 02:06:16,1,1,FORWARD
2,17/08/2022 02:06:16,1,1,FORWARD
3,17/08/2022 02:06:16,1,1,FORWARD
4,17/08/2022 02:06:16,1,1,FORWARD
...,...,...,...,...
978,17/08/2022 02:07:14,0,0,STOP
979,17/08/2022 02:07:14,0,0,STOP
980,17/08/2022 02:07:14,0,0,STOP
981,17/08/2022 02:07:14,0,0,STOP


In [6]:
sample8_images

,Unnamed: 0,image,time
0,0,0.jpg,2022-08-17 02:06:18.768545
1,1,1.jpg,2022-08-17 02:06:18.790247
2,2,2.jpg,2022-08-17 02:06:18.823686
3,3,3.jpg,2022-08-17 02:06:18.857676
4,4,4.jpg,2022-08-17 02:06:18.891395
...,...,...,...
1593,1593,1593.jpg,2022-08-17 02:07:11.944305
1594,1594,1594.jpg,2022-08-17 02:07:11.949379
1595,1595,1595.jpg,2022-08-17 02:07:11.955159
1596,1596,1596.jpg,2022-08-17 02:07:11.981563


Start -- 2:06:19
End -- 02:07:08

In [7]:
def trimDot(text, date = True):
    sep = '.'

    if(date == True):
        stripped = text.split(sep, 1)[0]
        stripped = stripped.replace('-', '/')
        return (stripped)
    else:
        stripped = text.split(sep, 1)[1][:4]
        stripped = float(stripped)/10000
        return stripped
    
    

In [8]:
def prepare_clean_datasamples(df, sample_images, sample_ir, start_timeStamp, end_timeStamp):
    sample_images['milliseconds'] = sample_images['time'].apply(trimDot, date=False)
    sample_images['trimmed_date'] = sample_images['time'].apply(trimDot)

    sample_images['trimmed_date'] = pd.to_datetime(sample_images['trimmed_date'])

    sample_ir['timeStamp'] = pd.to_datetime(sample8_ir['timeStamp'])

    sample_ir = sample_ir.loc[((sample_ir['timeStamp'] > start_timeStamp) & (sample_ir['timeStamp'] < end_timeStamp))]
    sample_images = sample_images.loc[((sample_images['trimmed_date'] > start_timeStamp) & (sample_images['trimmed_date'] < end_timeStamp))]

    unique_date_values = sample_images['trimmed_date'].unique()

    sample_ir.loc[sample_ir['timeStamp'] == start_timeStamp]

    for i in unique_date_values:
        unique_ir = sample_ir.loc[sample_ir['timeStamp'] == i]
        number_of_ir = unique_ir.shape[0]

        j = 0
        range_values = []
        while j < number_of_ir:
            range_values.append((1/number_of_ir)*j)
            j += 1

        label_idx = 0

        for k, single_range in enumerate(range_values):
            if k == 0:
                first_range = 0.0
                second_range = range_values[k]
            elif k == (len(range_values)) - 1:
                first_range = range_values[k]
                second_range = 1.0
            else:
                first_range = range_values[k]
                second_range = range_values[k+1]
            label = unique_ir.iloc[label_idx]['ACTION']
            label_idx += 1
            
            data = sample_images.loc[((sample_images['trimmed_date'] == i ) & (((sample_images['milliseconds']) > first_range) & ((sample_images['milliseconds']) < second_range)))]
            images = data['image'].values
            if(len(images) != 0):
                for image in (images):
                    data_point = {
                        'image' : image,
                        'label' : label
                    }
                    df = df.append(data_point, ignore_index=True)

In [9]:
start_timeStamp = '2022-08-17 02:06:19'
end_timeStamp = '2022-08-17 02:07:08'
prepare_clean_datasamples(df, sample_images=sample8_images, sample_ir=sample8_ir, start_timeStamp=start_timeStamp, end_timeStamp=end_timeStamp)

In [10]:
sample8_images['milliseconds'] = sample8_images['time'].apply(trimDot, date=False)
sample8_images['trimmed_date'] = sample8_images['time'].apply(trimDot)


In [11]:
sample8_images.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1598 entries, 0 to 1597
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Unnamed: 0    1598 non-null   int64  
 1   image         1598 non-null   object 
 2   time          1598 non-null   object 
 3   milliseconds  1598 non-null   float64
 4   trimmed_date  1598 non-null   object 
dtypes: float64(1), int64(1), object(3)
memory usage: 62.5+ KB


In [12]:
sample8_ir['timeStamp'] = pd.to_datetime(sample8_ir['timeStamp'])
sample1_ir = sample8_ir.loc[((sample8_ir['timeStamp'] > '2022-08-17 02:06:19') & (sample8_ir['timeStamp'] < '2022-08-17 02:07:08'))]

In [13]:
unique_date_values = sample8_images['trimmed_date'].unique()

In [14]:
df = pd.DataFrame(columns=['image', 'label'])

for i in unique_date_values:
    unique_ir = sample1_ir.loc[sample1_ir['timeStamp'] == i]
    number_of_ir = unique_ir.shape[0]

    j = 0
    range_values = []
    while j < number_of_ir:
        range_values.append((1/number_of_ir)*j)
        j += 1

    label_idx = 0
    
    for k, single_range in enumerate(range_values):
        if k == 0:
            first_range = 0.0
            second_range = range_values[k]
        elif k == (len(range_values)) - 1:
            first_range = range_values[k]
            second_range = 1.0
        else:
            first_range = range_values[k]
            second_range = range_values[k+1]
        label = unique_ir.iloc[label_idx]['ACTION']
        label_idx += 1
        data = sample8_images.loc[((sample8_images['trimmed_date'] == i ) & (((sample8_images['milliseconds']) > first_range) & ((sample8_images['milliseconds']) < second_range)))]

        images = data['image'].values
        if(len(images) != 0):
            for image in (images):
                data_point = {
                    'image' : image,
                    'label' : label
                }
                df = df.append(data_point, ignore_index=True)

KeyboardInterrupt: 

In [ ]:
df['label'].value_counts()

Series([], Name: label, dtype: int64)

In [ ]:
ir_per_second = {}
for i in range(10, 60):
    ir_no = (sample8_ir.loc[sample8_ir['timeStamp'] == f'2022-08-17 01:49:{i}']).shape[0]
    ir_per_second[i] = ir_no

In [ ]:
sample1_ir['timeStamp'].value_counts()[0: 10]

Series([], Name: timeStamp, dtype: int64)

In [ ]:
images_per_second = {}
for i in range(10, 60):
    images_no = (sample8_images.loc[sample8_images['time'] == f'2022-08-17 01:49:{i}']).shape[0]
    images_per_second[i] = images_no

In [ ]:
sample1_ir = pd.read_csv(r'C:\Users\wasim\line-following\Data Samples\DataSamples-1\data_ir.csv')
sample1_images = pd.read_csv(r'C:\Users\wasim\line-following\Data Samples\DataSamples-1\out_images.csv')

print(sample1_ir[-150: -100])
print("=" * 50)
print(sample1_images[0:40])

                timeStamp  IR1  IR2   ACTION
1387  17/08/2022 01:19:38    1    1  FORWARD
1388  17/08/2022 01:19:38    1    1  FORWARD
1389  17/08/2022 01:19:38    1    1  FORWARD
1390  17/08/2022 01:19:38    1    1  FORWARD
1391  17/08/2022 01:19:38    1    1  FORWARD
1392  17/08/2022 01:19:38    1    1  FORWARD
1393  17/08/2022 01:19:38    1    1  FORWARD
1394  17/08/2022 01:19:38    1    1  FORWARD
1395  17/08/2022 01:19:38    1    0    RIGHT
1396  17/08/2022 01:19:38    1    1  FORWARD
1397  17/08/2022 01:19:38    1    1  FORWARD
1398  17/08/2022 01:19:38    1    1  FORWARD
1399  17/08/2022 01:19:38    1    1  FORWARD
1400  17/08/2022 01:19:38    1    1  FORWARD
1401  17/08/2022 01:19:38    1    1  FORWARD
1402  17/08/2022 01:19:38    1    1  FORWARD
1403  17/08/2022 01:19:39    1    1  FORWARD
1404  17/08/2022 01:19:39    1    1  FORWARD
1405  17/08/2022 01:19:39    1    1  FORWARD
1406  17/08/2022 01:19:39    1    1  FORWARD
1407  17/08/2022 01:19:39    1    1  FORWARD
1408  17/0

Start -- 01:18:51
End -- 01:19:39

In [ ]:
sample2_ir = pd.read_csv(r'C:\Users\wasim\line-following\Data Samples\DataSamples-2\data_ir.csv')
sample2_images = pd.read_csv(r'C:\Users\wasim\line-following\Data Samples\DataSamples-2\out_images.csv')

print(sample2_ir[-50: -1])
print("=" * 50)
print(sample2_images[0:40])

                timeStamp  IR1  IR2   ACTION
1761  17/08/2022 01:50:51    1    1  FORWARD
1762  17/08/2022 01:50:51    1    1  FORWARD
1763  17/08/2022 01:50:51    1    1  FORWARD
1764  17/08/2022 01:50:51    1    0    RIGHT
1765  17/08/2022 01:50:51    1    1  FORWARD
1766  17/08/2022 01:50:51    1    1  FORWARD
1767  17/08/2022 01:50:51    1    1  FORWARD
1768  17/08/2022 01:50:51    1    1  FORWARD
1769  17/08/2022 01:50:51    1    1  FORWARD
1770  17/08/2022 01:50:51    1    1  FORWARD
1771  17/08/2022 01:50:51    1    1  FORWARD
1772  17/08/2022 01:50:51    1    1  FORWARD
1773  17/08/2022 01:50:51    1    1  FORWARD
1774  17/08/2022 01:50:52    1    0    RIGHT
1775  17/08/2022 01:50:52    1    0    RIGHT
1776  17/08/2022 01:50:52    1    1  FORWARD
1777  17/08/2022 01:50:52    0    1     LEFT
1778  17/08/2022 01:50:52    1    1  FORWARD
1779  17/08/2022 01:50:52    1    1  FORWARD
1780  17/08/2022 01:50:52    1    1  FORWARD
1781  17/08/2022 01:50:52    1    1  FORWARD
1782  17/0

01:49:10
01:50:54

In [ ]:
sample3_ir = pd.read_csv(r'C:\Users\wasim\line-following\Data Samples\DataSamples-3\data_ir.csv')
sample3_images = pd.read_csv(r'C:\Users\wasim\line-following\Data Samples\DataSamples-3\out_images.csv')

print(sample3_ir[-150: -100])
print("=" * 50)
print(sample3_images[0:40])

               timeStamp  IR1  IR2   ACTION
599  17/08/2022 01:58:04    0    1     LEFT
600  17/08/2022 01:58:05    1    1  FORWARD
601  17/08/2022 01:58:05    1    1  FORWARD
602  17/08/2022 01:58:05    1    1  FORWARD
603  17/08/2022 01:58:05    1    1  FORWARD
604  17/08/2022 01:58:05    1    1  FORWARD
605  17/08/2022 01:58:05    1    1  FORWARD
606  17/08/2022 01:58:05    1    1  FORWARD
607  17/08/2022 01:58:05    1    1  FORWARD
608  17/08/2022 01:58:05    1    1  FORWARD
609  17/08/2022 01:58:05    1    1  FORWARD
610  17/08/2022 01:58:05    1    0    RIGHT
611  17/08/2022 01:58:05    1    0    RIGHT
612  17/08/2022 01:58:05    1    1  FORWARD
613  17/08/2022 01:58:05    0    1     LEFT
614  17/08/2022 01:58:06    1    1  FORWARD
615  17/08/2022 01:58:06    1    1  FORWARD
616  17/08/2022 01:58:06    1    1  FORWARD
617  17/08/2022 01:58:06    1    1  FORWARD
618  17/08/2022 01:58:06    1    1  FORWARD
619  17/08/2022 01:58:06    1    1  FORWARD
620  17/08/2022 01:58:06    1   

Start -- 01:57:32
End -- 01:58:06

In [ ]:
sample4_ir = pd.read_csv(r'C:\Users\wasim\line-following\Data Samples\DataSamples-4\data_ir.csv')
sample4_images = pd.read_csv(r'C:\Users\wasim\line-following\Data Samples\DataSamples-4\out_images.csv')

print(sample4_ir[-100: -50])
print("=" * 50)
print(sample4_images[0:40])

               timeStamp  IR1  IR2   ACTION
650  17/08/2022 01:59:13    1    1  FORWARD
651  17/08/2022 01:59:13    1    1  FORWARD
652  17/08/2022 01:59:13    1    1  FORWARD
653  17/08/2022 01:59:13    1    1  FORWARD
654  17/08/2022 01:59:13    1    1  FORWARD
655  17/08/2022 01:59:13    1    1  FORWARD
656  17/08/2022 01:59:13    1    1  FORWARD
657  17/08/2022 01:59:13    1    1  FORWARD
658  17/08/2022 01:59:13    1    1  FORWARD
659  17/08/2022 01:59:13    0    1     LEFT
660  17/08/2022 01:59:13    0    0     STOP
661  17/08/2022 01:59:13    0    0     STOP
662  17/08/2022 01:59:13    0    0     STOP
663  17/08/2022 01:59:14    0    0     STOP
664  17/08/2022 01:59:14    0    0     STOP
665  17/08/2022 01:59:14    0    0     STOP
666  17/08/2022 01:59:14    0    0     STOP
667  17/08/2022 01:59:14    0    0     STOP
668  17/08/2022 01:59:14    0    0     STOP
669  17/08/2022 01:59:14    0    0     STOP
670  17/08/2022 01:59:14    0    0     STOP
671  17/08/2022 01:59:14    0   

Start -- 01:58:39
End -- 01:59:13

In [ ]:
sample5_ir = pd.read_csv(r'C:\Users\wasim\line-following\Data Samples\DataSamples-5\data_ir.csv')
sample5_images = pd.read_csv(r'C:\Users\wasim\line-following\Data Samples\DataSamples-5\out_images.csv')

print(sample5_ir[-100: -50])
print("=" * 50)
print(sample5_images[0:40])

               timeStamp  IR1  IR2   ACTION
876  17/08/2022 02:02:08    1    1  FORWARD
877  17/08/2022 02:02:08    1    1  FORWARD
878  17/08/2022 02:02:08    1    1  FORWARD
879  17/08/2022 02:02:08    1    0    RIGHT
880  17/08/2022 02:02:09    1    1  FORWARD
881  17/08/2022 02:02:09    1    1  FORWARD
882  17/08/2022 02:02:09    1    1  FORWARD
883  17/08/2022 02:02:09    1    1  FORWARD
884  17/08/2022 02:02:09    1    1  FORWARD
885  17/08/2022 02:02:09    1    1  FORWARD
886  17/08/2022 02:02:09    1    1  FORWARD
887  17/08/2022 02:02:09    1    1  FORWARD
888  17/08/2022 02:02:09    1    1  FORWARD
889  17/08/2022 02:02:09    1    1  FORWARD
890  17/08/2022 02:02:09    1    1  FORWARD
891  17/08/2022 02:02:09    1    0    RIGHT
892  17/08/2022 02:02:09    1    0    RIGHT
893  17/08/2022 02:02:09    1    1  FORWARD
894  17/08/2022 02:02:10    1    1  FORWARD
895  17/08/2022 02:02:10    1    1  FORWARD
896  17/08/2022 02:02:10    1    1  FORWARD
897  17/08/2022 02:02:10    1   

Start -- 2022-08-17 02:01:20
End -- 2022-08-17 02:02:10

In [ ]:
sample6_ir = pd.read_csv(r'C:\Users\wasim\line-following\Data Samples\DataSamples-6\data_ir.csv')
sample6_images = pd.read_csv(r'C:\Users\wasim\line-following\Data Samples\DataSamples-6\out_images.csv')

print(sample6_ir[-150: -90])
print("=" * 50)
print(sample6_images[0:40])

               timeStamp  IR1  IR2   ACTION
452  17/08/2022 02:04:09    1    1  FORWARD
453  17/08/2022 02:04:09    1    1  FORWARD
454  17/08/2022 02:04:09    0    1     LEFT
455  17/08/2022 02:04:09    1    1  FORWARD
456  17/08/2022 02:04:10    1    1  FORWARD
457  17/08/2022 02:04:10    1    1  FORWARD
458  17/08/2022 02:04:10    1    1  FORWARD
459  17/08/2022 02:04:10    1    1  FORWARD
460  17/08/2022 02:04:10    1    1  FORWARD
461  17/08/2022 02:04:10    1    1  FORWARD
462  17/08/2022 02:04:10    1    1  FORWARD
463  17/08/2022 02:04:10    1    1  FORWARD
464  17/08/2022 02:04:10    1    1  FORWARD
465  17/08/2022 02:04:10    1    1  FORWARD
466  17/08/2022 02:04:10    1    1  FORWARD
467  17/08/2022 02:04:10    1    1  FORWARD
468  17/08/2022 02:04:10    1    1  FORWARD
469  17/08/2022 02:04:10    1    1  FORWARD
470  17/08/2022 02:04:10    1    1  FORWARD
471  17/08/2022 02:04:10    1    0    RIGHT
472  17/08/2022 02:04:10    1    0    RIGHT
473  17/08/2022 02:04:11    1   

Start -- 02:03:45
End -- 02:04:12 

In [ ]:
sample7_ir = pd.read_csv(r'C:\Users\wasim\line-following\Data Samples\DataSamples-7\data_ir.csv')
sample7_images = pd.read_csv(r'C:\Users\wasim\line-following\Data Samples\DataSamples-7\out_images.csv')

print(sample7_ir.iloc[-1])
print("=" * 50)
print(sample7_images[0:40])

timeStamp    17/08/2022 02:05:46
IR1                            0
IR2                            1
ACTION                      LEFT
Name: 799, dtype: object
    Unnamed: 0   image                        time
0            0   0.jpg  2022-08-17 02:04:59.040421
1            1   1.jpg  2022-08-17 02:04:59.064482
2            2   2.jpg  2022-08-17 02:04:59.097891
3            3   3.jpg  2022-08-17 02:04:59.133784
4            4   4.jpg  2022-08-17 02:04:59.166059
5            5   5.jpg  2022-08-17 02:04:59.199715
6            6   6.jpg  2022-08-17 02:04:59.231748
7            7   7.jpg  2022-08-17 02:04:59.266115
8            8   8.jpg  2022-08-17 02:04:59.297331
9            9   9.jpg  2022-08-17 02:04:59.329872
10          10  10.jpg  2022-08-17 02:04:59.365367
11          11  11.jpg  2022-08-17 02:04:59.397194
12          12  12.jpg  2022-08-17 02:04:59.430402
13          13  13.jpg  2022-08-17 02:04:59.463933
14          14  14.jpg  2022-08-17 02:04:59.501088
15          15  15.jpg  202

Start -- 02:04:59
End -- 02:05:46

In [ ]:
df1.head()

NameError: name 'df1' is not defined